In [102]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import glob
import pickle



In [103]:
%matplotlib inline
# Load saved pickle data

dist_pickle = pickle.load(open("camera_cal/calibration_pickle.p","rb"))
dist = dist_pickle["dist"]
mtx = dist_pickle["mtx"]



In [104]:
class tracker():
    
    def __init__(self,mywindow_widh,mywindow_height,mymargin,my_ym=1,my_xm=1,my_smooth_factor=15):
        self.recent_centers=[]
        self.window_width=mywindow_widh
        self.window_height=mywindow_height
        self.margin=mymargin
        self.ym_per_pix=my_ym
        self.xm_per_pix=my_xm
        self.smooth_factor=my_smooth_factor
        
        
    
    def find_window_centroids(self,image):
    
        window_centroids = [] # Store the (left,right) window centroid positions per level
        window = np.ones(self.window_width) # Create our window template that we will use for convolutions

        # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
        # and then np.convolve the vertical image slice with the window template 

        # Sum quarter bottom of image to get slice, could use a different ratio
        l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
        l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
        r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
        r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)

        # Add what we found for the first layer
        window_centroids.append((l_center,r_center))

        # Go through each layer looking for max pixel locations
        for level in range(1,(int)(image.shape[0]/self.window_height)):
            # convolve the window into the vertical slice of the image
            image_layer = np.sum(image[int(image.shape[0]-(level+1)*self.window_height):int(image.shape[0]-level*window_height),:], axis=0)
            conv_signal = np.convolve(window, image_layer)
            # Find the best left centroid by using past left center as a reference
            # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
            offset = self.window_width/2
            l_min_index = int(max(l_center+offset-self.margin,0))
            l_max_index = int(min(l_center+offset+self.margin,image.shape[1]))
            l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
            # Find the best right centroid by using past right center as a reference
            r_min_index = int(max(r_center+offset-self.margin,0))
            r_max_index = int(min(r_center+offset+self.margin,image.shape[1]))
            r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
            # Add what we found for that layer
            window_centroids.append((l_center,r_center))
#             self.recent_centers = window_centroids
        return window_centroids



In [105]:
def window_mask(width,height,img_ref,center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

In [106]:
def abs_sobel_thresh(image, orient='x',sobel_kernel = 3,thresh=(0,255)):
    
    
    #     Convert to grayscale
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    #     Take the derivate in given x or y orient
    if orient=='x':
        sobel = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize=sobel_kernel)
    if orient=='y':
        sobel = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    
    # Calculate the absolute value of x derivate
    abs_sobel = np.absolute(sobel)
    
    #     scale to 8 bit
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    #     create a binary threshold
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel>=thresh[0]) & (scaled_sobel<thresh[1])]=1
    
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output
    
def color_thresh(image,sthresh=(0,255),vthresh=(0,255)):
    hls = cv2.cvtColor(image,cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= sthresh[0])&(s_channel <= sthresh[1])]=1
    
#     hsv = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
#     v_channel= hsv[:,:,2]
#     v_binary = np.zeros_like(v_channel)
#     v_binary[(v_channel >= vthresh[0]) & (v_channel < vthresh[1])]=1
    
    output = np.zeros_like(s_channel)
    
    output[(s_binary==1) ]=1
#     output[(s_binary==1) & (v_binary==1)]=1
    # Stack each channel
#     color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    
#     write_img = 'color_thresh_image'+str(idx)+'.jpg'
#     cv2.imwrite('output_images/'+write_img,output)
    
    
    return output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

    

    



In [111]:
# Read Images
images = glob.glob('test_images/test*.jpg')
print(len(images))
for idx,fname in enumerate(images):
    img= cv2.imread(fname)  
    img = cv2.undistort(img,mtx,dist,None,mtx)
 
    combined_binary = np.zeros_like(img[:,:,0])
    
    gradx = abs_sobel_thresh(img,orient='x',thresh=(20,100))
    grady = abs_sobel_thresh(img,orient='y',thresh=(25,255))
    mag_binary = mag_thresh(img)
    dir_binary = dir_threshold(img)
    
 
    c_binary = color_thresh(img,sthresh=(100,255),vthresh=(50,100))  
   
    combined_binary[(gradx==1) & (grady==1) | (mag_binary==1)&(dir_binary==1) |(c_binary==1) ]=1
    
    write_img = 'binary_image'+str(idx)+'.jpg'
    cv2.imwrite('examples/'+write_img,combined_binary)
    
       
   
    img_size = (img.shape[1],img.shape[0])
    bot_width = 0.76
    mid_width= 0.08
    height_pct = 0.62
    bottom_trim = 0.935
    src = np.float32([[img.shape[1]*(0.5-mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(0.5+mid_width/2),img.shape[0]*height_pct],[img.shape[1]*(0.5+bot_width/2),img.shape[0]*bottom_trim],[img.shape[1]*(0.5-bot_width/2),img.shape[0]*bottom_trim]])
    print(src)
    offset=img_size[0]*0.25
    dst = np.float32([[offset,0],[img_size[0]-offset,0],[img_size[0]-offset,img_size[1]],[offset,img_size[1]]])
    

#     src = np.float32(
#     [[(img_size[0] / 2) - 55, img_size[1] / 2 + 100],
#     [((img_size[0] / 6) - 10), img_size[1]],
#     [(img_size[0] * 5 / 6) + 60, img_size[1]],
#     [(img_size[0] / 2 + 55), img_size[1] / 2 + 100]])

#     dst = np.float32(
#     [[(img_size[0] / 4), 0],
#     [(img_size[0] / 4), img_size[1]],
#     [(img_size[0] * 3 / 4), img_size[1]],
#     [(img_size[0] * 3 / 4), 0]])
    
   
    M = cv2.getPerspectiveTransform(src,dst)
    Minv = cv2.getPerspectiveTransform(dst,src)
    warped_img = cv2.warpPerspective(combined_binary,M,img_size,flags= cv2.INTER_LINEAR)
    
    
#     write_img = 'warped_image'+str(idx)+'.jpg'
#     print(write_img)
#     cv2.imwrite('examples/'+write_img,warped_img)
    
    
    window_width=50
    window_height=80
    margin=50
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    curve_center = tracker(window_width,window_height,margin,xm_per_pix,ym_per_pix,15)
    window_centroids = curve_center.find_window_centroids(warped_img)
    print(window_centroids)
   
    
    # If we found any window centers
    if len(window_centroids) > 0:
        

        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped_img)
        r_points = np.zeros_like(warped_img)
        
        leftx=[]
        rightx=[]

        # Go through each level and draw the windows 	
        for level in range(0,len(window_centroids)):
            
            leftx.append(window_centroids[level][0])
            rightx.append(window_centroids[level][1])
            
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped_img,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped_img,window_centroids[level][1],level)
            
            # Add graphic points from window mask here to total pixels found 
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage= np.dstack((warped_img, warped_img, warped_img))*255 # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results

    # If no window centers found, just display orginal road image
    else:
        output = np.array(cv2.merge((warped_img,warped_img,warped_img)),np.uint8)

    write_img = 'lane_image'+str(idx)+'.jpg'
    print(write_img)
    cv2.imwrite('examples/'+write_img,output)
    
    
    yvals =range(0,warped_img.shape[0])
    
   
    res_vals = np.arange(warped_img.shape[0]-(window_height/2),0,-window_height)
  
    
    left_fit = np.polyfit(res_vals,leftx,2)
    left_fitx = left_fit[0]*yvals*yvals+left_fit[1]*yvals+left_fit[2]
    left_fitx = np.array(left_fitx,np.int32)
    
    right_fit= np.polyfit(res_vals,rightx,2)
    right_fitx= right_fit[0]*yvals*yvals+right_fit[1]*yvals+right_fit[2]
    right_fitx = np.array(right_fitx,np.int32)
    
    left_lane = np.array(list(zip(np.concatenate((left_fitx-window_width/2,left_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    right_lane = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
    middle_marker = np.array(list(zip(np.concatenate((right_fitx-window_width/2,right_fitx[::-1]+window_width/2),axis=0),np.concatenate((yvals,yvals[::-1]),axis=0))),np.int32)
   
    road = np.zeros_like(img)
    road_bkg = np.zeros_like(img)
    cv2.fillPoly(road,[left_lane],color=[255,0,0])
    cv2.fillPoly(road,[right_lane],color=[0,0,255])
    cv2.fillPoly(road_bkg,[left_lane],color=[255,255,255])
    cv2.fillPoly(road_bkg,[right_lane],color=[255,255,255])
    
    road_warped = cv2.warpPerspective(road,Minv,img_size,flags=cv2.INTER_LINEAR)
    road_bkg_warped = cv2.warpPerspective(road_bkg,Minv,img_size,flags=cv2.INTER_LINEAR)
    
    base = cv2.addWeighted(img,1.0,road_warped,-1.0,0.0)
    result = cv2.addWeighted(base,1.0,road_bkg_warped,1.0,0.0)
    
    xm_per_pix = curve_center.xm_per_pix
    ym_per_pix = curve_center.ym_per_pix
    
    curve_fit_cr = np.polyfit(np.array(res_vals,np.float32)*ym_per_pix,np.array(leftx,np.float32),2)
#     curve_fit_cr = np.polyfit(np.array(res_vals,np.float32)*ym_per_pix,np.array(leftx,np.float32),2)
    curverad = ((1+(2*curve_fit_cr[0]*yvals[-1]*ym_per_pix+curve_fit_cr[1])**2)**1.5)/(np.absolute(2*curve_fit_cr[0]))
    
    
    camera_center = (left_fitx[-1]+right_fitx[-1])/2
    center_diff = (camera_center-img.shape[1]/2)*xm_per_pix
    side_pos='left'
    if(center_diff<=0):
        side_pos='right'
    
    cv2.putText(result,'Center of curvature = '+str(round(curverad,3))+'(m)',(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
    cv2.putText(result,'Vehicle is '+str(round(center_diff,3)) + 'm' +side_pos+' of center ',(50,100),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
   
    name = "test_images/tracked"+str(idx)+".jpg"
   
    cv2.imwrite(name,result)
 
    

6
[[  588.79998779   446.3999939 ]
 [  691.20001221   446.3999939 ]
 [ 1126.40002441   673.20001221]
 [  153.6000061    673.20001221]]
[(238.0, 664.0), (188.0, 614.0), (138.0, 564.0), (88.0, 514.0), (38.0, 464.0), (24.0, 414.0), (24.0, 364.0), (24.0, 314.0), (24.0, 264.0)]
lane_image0.jpg
[[  588.79998779   446.3999939 ]
 [  691.20001221   446.3999939 ]
 [ 1126.40002441   673.20001221]
 [  153.6000061    673.20001221]]
[(238.0, 664.0), (188.0, 614.0), (138.0, 564.0), (88.0, 514.0), (38.0, 464.0), (24.0, 414.0), (24.0, 364.0), (24.0, 314.0), (24.0, 264.0)]
lane_image1.jpg
[[  588.79998779   446.3999939 ]
 [  691.20001221   446.3999939 ]
 [ 1126.40002441   673.20001221]
 [  153.6000061    673.20001221]]
[(238.0, 664.0), (188.0, 614.0), (138.0, 564.0), (88.0, 514.0), (38.0, 464.0), (24.0, 414.0), (24.0, 364.0), (24.0, 314.0), (24.0, 264.0)]
lane_image2.jpg
[[  588.79998779   446.3999939 ]
 [  691.20001221   446.3999939 ]
 [ 1126.40002441   673.20001221]
 [  153.6000061    673.20001221]]
[